# Module 

In [36]:
! pip list | grep xg

xgboost                           1.5.2


In [6]:


# from __future__ import annotations
from time import time
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import xgboost as xgb






In [32]:

def load_cat_in_the_dat() -> Tuple[pd.DataFrame, pd.Series]:
    """Assuming you have already downloaded the data into `input` directory."""

    df_train = pd.read_csv("../data/train.csv")

    print(
        "train data set has got {} rows and {} columns".format(
            df_train.shape[0], df_train.shape[1]
        )
    )
    X = df_train.drop(["target"], axis=1)
    y = df_train["target"]

    for i in range(0, 5):
        X["bin_" + str(i)] = X["bin_" + str(i)].astype("category")

    for i in range(0, 5):
        X["nom_" + str(i)] = X["nom_" + str(i)].astype("category")

    for i in range(5, 10):
        X["nom_" + str(i)] = X["nom_" + str(i)].apply(int, base=16)

    for i in range(0, 6):
        X["ord_" + str(i)] = X["ord_" + str(i)].astype("category")

    print(
        "train data set has got {} rows and {} columns".format(X.shape[0], X.shape[1])
    )
    return X, y

In [33]:
params = {
    "tree_method": "hist",
    "use_label_encoder": False,
    "n_estimators": 32,
    "colsample_bylevel": 0.7,
}


def categorical_model(X: pd.DataFrame, y: pd.Series, output_dir: str) -> None:
    """Train using builtin categorical data support from XGBoost"""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1994, test_size=0.2
    )
    # Specify `enable_categorical` to True.
    clf = xgb.XGBClassifier(
        **params,
        eval_metric="auc",
        enable_categorical=True,
        max_cat_to_onehot=1,  # We use optimal partitioning exclusively
    )
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)])
    clf.save_model(os.path.join(output_dir, "categorical.json"))

    y_score = clf.predict_proba(X_test)[:, 1]  # proba of positive samples
    auc = roc_auc_score(y_test, y_score)
    print("AUC of using builtin categorical data support:", auc)


def onehot_encoding_model(X: pd.DataFrame, y: pd.Series, output_dir: str) -> None:
    """Train using one-hot encoded data."""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=42, test_size=0.2
    )
    # Specify `enable_categorical` to False as we are using encoded data.
    clf = xgb.XGBClassifier(**params, eval_metric="auc", enable_categorical=False)
    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test), (X_train, y_train)],
    )
    clf.save_model(os.path.join(output_dir, "one-hot.json"))

    y_score = clf.predict_proba(X_test)[:, 1]  # proba of positive samples
    auc = roc_auc_score(y_test, y_score)
    print("AUC of using onehot encoding:", auc)



In [35]:
X, y = load_cat_in_the_dat()

with TemporaryDirectory() as tmpdir:
    start = time()
    categorical_model(X, y, tmpdir)
    end = time()
    print("Duration:categorical", end - start)

#     X = pd.get_dummies(X)
#     start = time()
#     onehot_encoding_model(X, y, tmpdir)
#     end = time()
#     print("Duration:onehot", end - start)

train data set has got 300000 rows and 25 columns
train data set has got 300000 rows and 24 columns


ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.bin_0, bin_1, bin_2, bin_3, bin_4, nom_0, nom_1, nom_2, nom_3, nom_4, ord_0, ord_1, ord_2, ord_3, ord_4, ord_5

In [37]:
! pip list | grep xg

xgboost                           1.5.2


In [29]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 258467 to 191843
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   id      240000 non-null  int64   
 1   bin_0   240000 non-null  category
 2   bin_1   240000 non-null  category
 3   bin_2   240000 non-null  category
 4   bin_3   240000 non-null  category
 5   bin_4   240000 non-null  category
 6   nom_0   240000 non-null  category
 7   nom_1   240000 non-null  category
 8   nom_2   240000 non-null  category
 9   nom_3   240000 non-null  category
 10  nom_4   240000 non-null  category
 11  nom_5   240000 non-null  int64   
 12  nom_6   240000 non-null  int64   
 13  nom_7   240000 non-null  int64   
 14  nom_8   240000 non-null  int64   
 15  nom_9   240000 non-null  int64   
 16  ord_0   240000 non-null  category
 17  ord_1   240000 non-null  category
 18  ord_2   240000 non-null  category
 19  ord_3   240000 non-null  category
 20  ord_4   240000 non-nu

In [28]:
clf.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)])


ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.bin_0, bin_1, bin_2, bin_3, bin_4, nom_0, nom_1, nom_2, nom_3, nom_4, ord_0, ord_1, ord_2, ord_3, ord_4, ord_5

In [20]:
X, y = load_cat_in_the_dat()

with TemporaryDirectory() as tmpdir:
#     start = time()
#     categorical_model(X, y, tmpdir)
#     end = time()
#     print("Duration:categorical", end - start)

    X = pd.get_dummies(X)
    start = time()
    onehot_encoding_model(X, y, tmpdir)
    end = time()
    print("Duration:onehot", end - start)

train data set has got 300000 rows and 25 columns
train data set has got 300000 rows and 24 columns
[21:26:46] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { enable_categorical } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.65091	validation_1-auc:0.65672
[1]	validation_0-auc:0.67295	validation_1-auc:0.67806
[2]	validation_0-auc:0.67826	validation_1-auc:0.68429
[3]	validation_0-auc:0.68405	validation_1-auc:0.68979
[4]	validation_0-auc:0.69325	validation_1-auc:0.69976
[5]	validation_0-auc:0.69648	validation_1-auc:0.70385
[6]	validation_0-auc:0.70192	validation_1-auc:0.70995
[7]	validation_0-auc:0.70513	validation_1-auc:0.71402
[8]	validation_0-auc: